In [3]:
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import ConversationalRetrievalChain

# Load environment variables
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# Model and database setup
MODEL = "gpt-4o-mini"
db_name = "vector_db"

# Create knowledge-base directory and move InsuranceCompany_Details.md
os.makedirs("knowledge-base/insurance", exist_ok=True)
with open("knowledge-base/insurance/InsuranceCompany_Details.md", "w", encoding="utf-8") as f:
    f.write("""
# InsuranceCompany Overview

## Overview
InsuranceCompany, founded in 2005 and headquartered in Mumbai, India, is a leading provider of insurance solutions. Led by CEO Priya Sharma and COO Vikram Desai, the company leverages advanced technology to deliver seamless customer experiences and serves a growing customer base across India.

## Business Metrics
- **Valuation**: $2.3 billion (2023, Series C funding of $150 million led by Global Ventures).
- **Revenue**: FY24 net profit of ₹80 crore (3x growth from FY23).
- **Total Policies Issued**: 1.2 million active policies as of 2024.
- **Customers**: Serves 800,000+ individuals and 50,000+ businesses.

## Innovations
- First Indian insurer to offer AI-driven claim processing, reducing settlement time to 24 hours.
- Introduced blockchain-based policy management for transparency.
- Launched a mobile app with real-time policy tracking and claims filing.

## Funding and Investors
Backed by Sequoia Capital India, SoftBank Vision Fund, Accel, and HDFC Bank.

## Strategic Focus
- Expanding digital-first insurance products for rural markets.
- Enhancing AI and IoT for risk assessment and fraud detection.
- Targeting 2 million policies by 2026.

## Recent Developments
- Received IRDAI approval for a new micro-insurance product in 2024.
- Partnered with TechMahindra for AI-driven customer support in 2023.
- Launched a sustainability initiative to offset carbon emissions from operations.

## Mission
To provide affordable, reliable, and innovative insurance solutions, empowering customers to secure their future with confidence.

## Contact
- Website: https://insurancecompany.com
- Headquarters: Mumbai, Maharashtra, India
- Employee Count: ~4,500 (2024)
""")

# Define add_metadata function
def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# Read in documents, ensuring only InsuranceCompany content is processed
folders = glob.glob("knowledge-base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    # Skip any folders that might contain Razorpay data
    if doc_type.lower() != "razorpay":
        loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
        folder_docs = loader.load()
        documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

# Create vector store
embeddings = OpenAIEmbeddings()
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

# Set up LLM
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Set up conversation history
class CustomChatHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_message(self, message):
        self.messages.append(message)

    def clear(self):
        self.messages = []

# Create a session-aware conversation chain
conversational_retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 25})
)

# Wrap with message history
def get_session_history(session_id: str) -> CustomChatHistory:
    if f"history_{session_id}" not in get_session_history.store:
        get_session_history.store[f"history_{session_id}"] = CustomChatHistory()
    return get_session_history.store[f"history_{session_id}"]

get_session_history.store = {}

conversation_with_history = RunnableWithMessageHistory(
    conversational_retrieval_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

# Chat function
def chat(question, history):
    # Filter out any Razorpay-related queries
    if "razorpay" in question.lower():
        return "I'm sorry, I can only provide details about InsuranceCompany. Please ask about InsuranceCompany instead."
    response = conversation_with_history.invoke(
        {"question": question},
        config={"configurable": {"session_id": "user_1"}}  # Unique session ID
    )
    return response["answer"]

# Launch Gradio interface
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

Total number of chunks: 2
Document types found: {'insurance'}
Vectorstore created with 2 documents
* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Number of requested results 25 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 25 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 25 is greater than number of elements in index 2, updating n_results = 2
